In [10]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import re

# Function to convert DMS to Decimal Degrees
def dms_to_dd(dms_str):
    match = re.match(r"(\d+)° (\d+)' ([\d.]+)\" ([NSEW])", dms_str.strip())
    if not match:
        raise ValueError(f"Invalid DMS format: {dms_str}")
    degrees, minutes, seconds, direction = match.groups()
    dd = float(degrees) + float(minutes)/60 + float(seconds)/3600
    if direction in ['S', 'W']:
        dd *= -1
    return dd

# Load the CSV file
df_global = pd.read_csv("Data/Sites_20250725_Global.csv", encoding='ISO-8859-1')
df_linz = pd.read_csv("Data/Sites_20250725_LINZ.csv", encoding='ISO-8859-1')

# Convert Latitude and Longitude from DMS to Decimal Degrees
df_global['latitude_dd'] = df_global['Latitude'].apply(dms_to_dd)
df_global['longitude_dd'] = df_global['Longitude'].apply(dms_to_dd)
df_linz['latitude_dd'] = df_linz['Latitude'].apply(dms_to_dd)
df_linz['longitude_dd'] = df_linz['Longitude'].apply(dms_to_dd)

# Create geometry column
geometry_global = [Point(xy) for xy in zip(df_global['longitude_dd'], df_global['latitude_dd'])]
geometry_linz = [Point(xy) for xy in zip(df_linz['longitude_dd'], df_linz['latitude_dd'])]

# Create GeoDataFrame
gdf_global = gpd.GeoDataFrame(df_global, geometry=geometry_global)
gdf_linz = gpd.GeoDataFrame(df_linz, geometry=geometry_linz)

# Set the coordinate ref syst
gdf_global.set_crs(epsg=4326, inplace=True)
gdf_linz.set_crs(epsg=4326, inplace=True)

# Display first few rows
print(gdf_global.head())
print(gdf_linz.head())

'''
# Optional: Save to GeoJSON or Shapefile
gdf_global.to_file("GeoJson/Sites_20250725_Global.geojson", driver="GeoJSON")
gdf_linz.to_file("GeoJson/Sites_20250725_LINZ.geojson", driver="GeoJSON")
#gdf.to_file("globalsites.shp")

print("GeoJSON 'Sites_20250725_Global.GeoJSON' has been successfully created.")
print("GeoJSON 'Sites_20250725_LINZ.GeoJSON' has been successfully created.")

# Export to GeoPackage
gdf_global.to_file("Geopackage Files/Sites_20250725_Global.gpkg", layer='sites', driver="GPKG")
gdf_linz.to_file("Geopackage Files/Sites_20250725_LINZ.gpkg", layer='sites', driver="GPKG")

print("GeoPackage 'Sites_20250725_Global.gpkg' has been successfully created.")
print("GeoPackage 'Sites_20250725_LINZ.gpkg' has been successfully created.")

'''

  Site Name Site Code  Ref Station Site Server Name  Sat. Sys.  \
0      GSKK      GSKK          108     Global Sites  G/R/E/C/J   
1      GSWR      GSWR           81     Global Sites  G/R/E/C/-   
2      GSDR      GSDR           80     Global Sites  G/R/E/C/-   
3      GSMH      GSMH           49     Global Sites  G/R/E/C/-   
4      GSSB      GSSB           66     Global Sites  G/R/E/C/J   

              Latitude             Longitude    Height  \
0  35° 13' 46.72650" S  173° 56' 56.48081" E  119.2490   
1  35° 43' 20.83912" S  174° 18' 54.55203" E   56.2220   
2  35° 56' 03.95319" S  173° 52' 52.96078" E   49.0896   
3  36° 05' 55.94327" S  174° 35' 08.74510" E   81.8460   
4  36° 34' 18.38774" S  174° 22' 39.40167" E   68.1860   

  Use Physical Coordinates    Physical Latitude  ... Physical Height  \
0                      Yes  35° 13' 46.69339" S  ...        119.2286   
1                      Yes  35° 43' 20.80646" S  ...         56.2128   
2                      Yes  35° 56' 03